# Assignment 2: Convolutional Autoencoders

Britt Schmitz  - i6235053 <br>
Tabea Heusel - i6323791

## 0 Setup

### 0.1 Imports

In [51]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

### 0.2 Loading Dataset

In [18]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
X = np.append(x_train, x_test, axis=0)
y = np.append(y_train, y_test, axis=0)

## 1 Autoencoder Network Implementation

### 1.1 Train, Validation, and Test 

In [34]:
# normalize
X = X / 255

# create train, validation, and test splits
X_train, X_remaining, y_train, y_remaining = train_test_split(X, y, test_size=0.2, random_state=3)
X_val, X_test, y_val, y_test = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=3)

### 1.2 Autoencoder Network

In [58]:
def create_cae():
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Conv2D(8, 3, 1, "same", activation="relu", input_shape=(32, 32, 3)))
    model.add(tf.keras.layers.MaxPool2D(2, padding="same"))
    model.add(tf.keras.layers.Conv2D(12, 3, 1, "same", activation="relu"))
    model.add(tf.keras.layers.MaxPool2D(2, padding="same"))
    model.add(tf.keras.layers.Conv2D(16, 3, 1, "same", activation="relu"))
    model.add(tf.keras.layers.UpSampling2D(2))
    model.add(tf.keras.layers.Conv2D(12, 3, 1, "same", activation="relu"))
    model.add(tf.keras.layers.UpSampling2D(2))
    model.add(tf.keras.layers.Conv2D(3, 3, 1, "same", activation="relu"))
    
    return model

In [ ]:
network = create_cae()
network.compile(optimizer="sgd", loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
history = network.fit(X_train, X_train, epochs=50, validation_data=(X_val, X_val))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('Binary Crossentropy')
plt.title('Evolution of Error During Training')
plt.legend(['train', 'validation'], loc='upper left')

Epoch 1/50
 341/1500 [=====>........................] - ETA: 43s - loss: 0.6512 - accuracy: 0.3725

In [ ]:
results = network.evaluate(X_test, X_test)
print(f"Test accuracy: {results[1]} \n Test error : {results[0]}")

In [ ]:
predictions = network.predict(X_test)

plt.imshow(X_test[5])
plt.show()
plt.imshow(predictions[5])
plt.show()